In [1]:
import numpy as np

IMG_PX_SIZE = 80
HM_SLICES = 16
LR = 1e-3

MODEL_NAME = 'boldvst1w-{}-{}.model.tflearn'.format(LR, '2-conv')


In [2]:
train_data = np.load('muchdata-80-80-16-smalldata.npy')

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None,IMG_PX_SIZE,IMG_PX_SIZE,1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR,loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [3]:
train = train_data[:-400]
test = train_data[-400:]

In [4]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_PX_SIZE,IMG_PX_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, IMG_PX_SIZE,IMG_PX_SIZE, 1)
test_y = [i[1] for i in test]

In [5]:
model.fit({'input':X},{'targets': Y}, n_epoch=3, validation_set=({'input':test_x},{'targets':test_y}),
         snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 416  | total loss: 3.41497 | time: 289.280s
| Adam | epoch: 003 | loss: 3.41497 - acc: 0.8512 -- iter: 8832/8880
Training Step: 417  | total loss: 3.43341 | time: 295.105s
| Adam | epoch: 003 | loss: 3.43341 - acc: 0.8504 | val_loss: 9.16438 - val_acc: 0.6000 -- iter: 8880/8880
--


In [6]:
model.save('model2.tflearn')

INFO:tensorflow:E:\brain\model2.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
tensorboard --logdir=foo:E:\brain\log

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
from tqdm import tqdm

IMG_PX_SIZE = 80

data_dir = 'C:/users/setcodestofire/documents/mygithub/brain/Image'
dataset  = os.listdir(data_dir)

In [ ]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l)

def process_data(data):
    
    path = data_dir +'/'+ str(data)
    img3d = nib.load(path)
    a = np.array(img3d.get_data())
    
    slice_img=[]
    for i in range(a.shape[2]):
        slice_img.append(cv2.resize(np.array(a[:,:,i]),(IMG_PX_SIZE,IMG_PX_SIZE)))
    
    
    new_slices = []
    
    chunk_sizes = math.ceil(len(slice_img) / 16)
    for slice_chunk in chunks(slice_img, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
    
    return np.array(new_slices)

img_data = process_data(dataset[0])

In [ ]:
model.predict(img_data[2].reshape(-1, 80,80, 1))